In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
from pathlib import Path 
import pycountry_convert as pc
import country_converter as coco

In [2]:
# Load the CSV into dataframe
advertising_df = pd.read_csv(os.path.join("Data", "Advertising_Data.csv"))
advertising_df.head()

,date,country,operator,channel,bid_timeouts_rate,render_rate,fillrate,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,avg_winning_bid (‚Ç¨),impressions,avg_imp_ecpm (‚Ç¨),revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
0,28-02-22,GB,Op_003,Channel_003,0.01,0.04,0.04,0.03,14,1818,1102,132,4.26,5,21.27,0.11,122.0,119.07,4.00
1,28-02-22,ES,Op_003,Channel_003,0.01,0.05,0.01,0.02,82,11312,506,162,14.96,8,36.28,0.14,244.0,80.20,4.00
2,28-02-22,FR,Op_003,Channel_003,0.00,0.06,0.01,0.03,19,2621,132,33,4.42,2,8.36,0.02,122.0,39.32,3.16
3,28-02-22,IT,Op_003,Channel_003,0.01,0.02,0.01,0.02,14,1932,83,47,3.46,1,10.87,0.01,122.0,85.71,2.14
4,28-02-22,DE,Op_003,Channel_003,0.01,0.00,0.00,0.00,1,139,13,5,0.00,0,0.00,0.00,122.0,122.00,0.00


In [3]:
# Make a copy of the dataframe
advertising_copy_df = advertising_df.copy(deep=True)
print(len(advertising_copy_df))

42045


In [4]:
# Drop unnecessary columns (from Client)
advertising_copy_df = advertising_copy_df.drop(['bid_timeouts_rate', 'render_rate', 'fillrate',
    'avg_winning_bid (‚Ç¨)', 'avg_imp_ecpm (‚Ç¨)'], axis=1)
advertising_copy_df.head()

,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
0,28-02-22,GB,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00
1,28-02-22,ES,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00
2,28-02-22,FR,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16
3,28-02-22,IT,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14
4,28-02-22,DE,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00


In [5]:
print(advertising_copy_df.columns)

Index(['date', 'country', 'operator', 'channel', 'pod_fill_rate',
       'endpoint_requests', 'bid_requests', 'bid_responses', 'bid_wons',
       'impressions', 'revenue', 'requested_pod_duration',
       'filled_pod_duration', 'delivered_pod_duration'],
      dtype='object')


In [6]:
# Check dtypes
advertising_copy_df.dtypes

date                       object
country                    object
operator                   object
channel                    object
pod_fill_rate             float64
endpoint_requests           int64
bid_requests                int64
bid_responses               int64
bid_wons                    int64
impressions                 int64
revenue                   float64
requested_pod_duration    float64
filled_pod_duration       float64
delivered_pod_duration    float64
dtype: object

In [7]:
# Convert column A to date format
advertising_copy_df['date'] = pd.to_datetime(advertising_copy_df['date'], dayfirst=True, errors='raise')
advertising_copy_df.head()

,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
0,2022-02-28,GB,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00
1,2022-02-28,ES,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00
2,2022-02-28,FR,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16
3,2022-02-28,IT,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14
4,2022-02-28,DE,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00


In [8]:
# Check dtypes again
advertising_copy_df.dtypes

date                      datetime64[ns]
country                           object
operator                          object
channel                           object
pod_fill_rate                    float64
endpoint_requests                  int64
bid_requests                       int64
bid_responses                      int64
bid_wons                           int64
impressions                        int64
revenue                          float64
requested_pod_duration           float64
filled_pod_duration              float64
delivered_pod_duration           float64
dtype: object

In [9]:
# Find rows where all values are zero except [endpoint_requests]
dirty_rows_df = advertising_copy_df.query('pod_fill_rate==0 & endpoint_requests!=0 & bid_requests==0 & bid_responses==0 & bid_wons==0 & impressions==0 & revenue==0 & requested_pod_duration==0 & filled_pod_duration==0 & delivered_pod_duration==0')
print(len(dirty_rows_df))
dirty_rows_df.head()

853


,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
1148,2022-06-23,NaN,No viewership data,No viewership data,0.0,4,0,0,0,0,0.0,0.0,0.0,0.0
1150,2022-06-24,NaN,No viewership data,No viewership data,0.0,18,0,0,0,0,0.0,0.0,0.0,0.0
1152,2022-06-25,NaN,No viewership data,No viewership data,0.0,7,0,0,0,0,0.0,0.0,0.0,0.0
1154,2022-06-26,NaN,No viewership data,No viewership data,0.0,15,0,0,0,0,0.0,0.0,0.0,0.0
1156,2022-06-27,NaN,No viewership data,No viewership data,0.0,5,0,0,0,0,0.0,0.0,0.0,0.0


In [10]:
# Add dirty indexes o a list
dirty_indexes = dirty_rows_df.index.array
dirty_indexes

<PandasArray>
[ 1148,  1150,  1152,  1154,  1156,  1158,  1161,  1163,  1165,  1167,
 ...
 41812, 41813, 41814, 41815, 41816, 41817, 41818, 41819, 41820, 41822]
Length: 853, dtype: int64

In [11]:
# Drop dirty rows
clean_advertising_df = advertising_copy_df.drop(dirty_indexes)

In [12]:
# Make sure the count is right
print(f'The clean dataframe has {len(clean_advertising_df.index)} rows')
print(f'There were {len(dirty_rows_df.index)} dirty rows to be removed')
print(f'The initial dataframe had {len(advertising_copy_df.index)} rows')
if len(clean_advertising_df.index) + len(dirty_rows_df.index) == len(advertising_copy_df.index):
    print('The math is right!!')
else:
    print('Ooops!')

The clean dataframe has 41192 rows
There were 853 dirty rows to be removed
The initial dataframe had 42045 rows
The math is right!!


In [13]:
# Find location of NaN values
missing_cols, missing_rows = (
    (clean_advertising_df.isnull().sum(x) | clean_advertising_df.eq('').sum(x))
    .loc[lambda x: x.gt(0)].index
    for x in (0,1)
)

print(f'There are {clean_advertising_df.loc[missing_rows, missing_cols].shape[0]} rows with empty cells across {clean_advertising_df.loc[missing_rows, missing_cols].shape[1]} columns')
almost_there_df = clean_advertising_df.loc[missing_rows, missing_cols]
almost_there_df.head(12)

There are 12 rows with empty cells across 1 columns


,country
15679,NaN
15994,NaN
16952,NaN
17018,NaN
22135,NaN
22742,NaN
28026,NaN
37785,NaN
39139,NaN
39299,NaN


In [14]:
# Add row indexes of NaN values to a list
dirty_indexes2 = almost_there_df.index.array
dirty_indexes2

<PandasArray>
[15679, 15994, 16952, 17018, 22135, 22742, 28026, 37785, 39139, 39299, 39316,
 39362]
Length: 12, dtype: int64

In [15]:
# Drop lines with NaN values in the 'country' column (only 12 values)
super_clean_df = clean_advertising_df.drop(dirty_indexes2)
print(super_clean_df.index)
super_clean_df.head()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            42035, 42036, 42037, 42038, 42039, 42040, 42041, 42042, 42043,
            42044],
           dtype='int64', length=41180)


,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
0,2022-02-28,GB,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00
1,2022-02-28,ES,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00
2,2022-02-28,FR,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16
3,2022-02-28,IT,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14
4,2022-02-28,DE,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00


In [16]:
# Make sure the count is right
print(f'The super_clean dataframe has {len(super_clean_df.index)} rows')
print(f'There were {len(almost_there_df.index)} dirty rows to be removed')
print(f'The clean dataframe had {len(clean_advertising_df.index)} rows')
if len(super_clean_df.index) + len(almost_there_df.index) == len(clean_advertising_df.index):
    print('The math is right!!')
else:
    print('Ooops!')

The super_clean dataframe has 41180 rows
There were 12 dirty rows to be removed
The clean dataframe had 41192 rows
The math is right!!


In [17]:
# Replace potential KeyErrors in the country code column
super_clean_df['country'] = super_clean_df['country'].replace("SX", "NL")
super_clean_df['country'] = super_clean_df['country'].replace("VA", "IT")

In [18]:
# Create a function to convert country names to matching continent names
def convert(row):
    try:
        continent_code = pc.country_alpha2_to_continent_code(row.country)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        print(f'{row.country} not found')

In [19]:
# Create a [region] column based on country names with pycountry
super_clean_df['region'] = super_clean_df.apply(convert, axis=1)
        
super_clean_df.head()

,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration,region
0,2022-02-28,GB,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00,Europe
1,2022-02-28,ES,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00,Europe
2,2022-02-28,FR,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16,Europe
3,2022-02-28,IT,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14,Europe
4,2022-02-28,DE,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00,Europe


In [20]:
# Convert country codes to country names
cc = coco.CountryConverter()
super_clean_df['country'] = cc.convert(names=super_clean_df['country'], to='name')
super_clean_df.head()

,date,country,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration,region
0,2022-02-28,United Kingdom,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00,Europe
1,2022-02-28,Spain,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00,Europe
2,2022-02-28,France,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16,Europe
3,2022-02-28,Italy,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14,Europe
4,2022-02-28,Germany,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00,Europe


In [21]:
# Check country names
super_clean_df['country'].unique()

array(['United Kingdom', 'Spain', 'France', 'Italy', 'Germany',
       'Switzerland', 'Austria', 'Isle of Man', 'India', 'Morocco',
       'Netherlands', 'United States', 'Ireland', 'China', 'Guatemala',
       'Sierra Leone', 'Jersey', 'Vietnam', 'Bulgaria', 'Puerto Rico',
       'Colombia', 'Pakistan', 'Gibraltar', 'United Arab Emirates',
       'Romania', 'Iraq', 'South Korea', 'Slovenia', 'Singapore',
       'Russia', 'Brazil', 'Monaco', 'Turkey', 'Serbia', 'Nigeria',
       'Poland', 'Andorra', 'Denmark', 'Egypt', 'Saudi Arabia',
       'Hong Kong', 'Canada', 'Armenia', 'Malta', 'Norway', 'Thailand',
       'Sweden', 'Tunisia', 'French Guiana', 'Ukraine', 'Czech Republic',
       'Paraguay', 'Kenya', 'Algeria', 'Reunion', 'Indonesia', 'Honduras',
       'Slovakia', 'Greece', 'Mexico', 'Panama', 'Iran', 'Belgium',
       'Peru', 'Bangladesh', 'Jordan', "Cote d'Ivoire", 'North Macedonia',
       'Lebanon', 'El Salvador', 'Kuwait', 'Qatar', 'Cameroon', 'Yemen',
       'Palestine', 'S

In [22]:
# Move the [region] column
super_clean_df = super_clean_df[['date', 'country', 'region', 'operator', 'channel', 'pod_fill_rate', 'endpoint_requests', 'bid_requests', 'bid_responses', 'bid_wons', 'impressions', 'revenue', 'requested_pod_duration', 'filled_pod_duration', 'delivered_pod_duration']]
super_clean_df.head()

,date,country,region,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,requested_pod_duration,filled_pod_duration,delivered_pod_duration
0,2022-02-28,United Kingdom,Europe,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,122.0,119.07,4.00
1,2022-02-28,Spain,Europe,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,244.0,80.20,4.00
2,2022-02-28,France,Europe,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,122.0,39.32,3.16
3,2022-02-28,Italy,Europe,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,122.0,85.71,2.14
4,2022-02-28,Germany,Europe,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,122.0,122.00,0.00


In [25]:
# Create new columns for [CPM] and pod drop rates
super_clean_df['CPM'] = super_clean_df['revenue'] *1000 / super_clean_df['impressions']
super_clean_df['requested_pod_droprate'] = ( super_clean_df['filled_pod_duration'] - super_clean_df['requested_pod_duration'] ) / super_clean_df['requested_pod_duration']
super_clean_df['filled_pod_droprate'] = ( super_clean_df['delivered_pod_duration'] - super_clean_df['filled_pod_duration'] ) / super_clean_df['filled_pod_duration']

# Rearrange columns
super_clean_df = super_clean_df[['date', 'country', 'region', 'operator', 'channel', 'pod_fill_rate', 'endpoint_requests', 'bid_requests', 'bid_responses', 'bid_wons', 'impressions', 'revenue', 'CPM', 'requested_pod_duration', 'requested_pod_droprate', 'filled_pod_duration', 'filled_pod_droprate', 'delivered_pod_duration']]

super_clean_df.head()

,date,country,region,operator,channel,pod_fill_rate,endpoint_requests,bid_requests,bid_responses,bid_wons,impressions,revenue,CPM,requested_pod_duration,requested_pod_droprate,filled_pod_duration,filled_pod_droprate,delivered_pod_duration
0,2022-02-28,United Kingdom,Europe,Op_003,Channel_003,0.03,14,1818,1102,132,5,0.11,22.0,122.0,-0.024016,119.07,-0.966406,4.00
1,2022-02-28,Spain,Europe,Op_003,Channel_003,0.02,82,11312,506,162,8,0.14,17.5,244.0,-0.671311,80.20,-0.950125,4.00
2,2022-02-28,France,Europe,Op_003,Channel_003,0.03,19,2621,132,33,2,0.02,10.0,122.0,-0.677705,39.32,-0.919634,3.16
3,2022-02-28,Italy,Europe,Op_003,Channel_003,0.02,14,1932,83,47,1,0.01,10.0,122.0,-0.297459,85.71,-0.975032,2.14
4,2022-02-28,Germany,Europe,Op_003,Channel_003,0.00,1,139,13,5,0,0.00,NaN,122.0,0.000000,122.00,-1.000000,0.00


In [34]:
# Review values in the newly-created columns
missing_cols, missing_rows = (
    (super_clean_df.isnull().sum(x) | super_clean_df.eq('').sum(x))
    .loc[lambda x: x.gt(0)].index
    for x in (0,1)
)

print(f'There are {super_clean_df.loc[missing_rows, missing_cols].shape[0]} rows with empty cells across {super_clean_df.loc[missing_rows, missing_cols].shape[1]} columns')
really_almost_there_df = super_clean_df.loc[missing_rows, missing_cols]
really_almost_there_df.head()

There are 18400 rows with empty cells across 3 columns


,CPM,requested_pod_droprate,filled_pod_droprate
4,NaN,0.000000,-1.0
8,NaN,-1.000000,NaN
11,NaN,-0.933770,-1.0
12,NaN,-0.508197,-1.0
13,NaN,-0.713115,-1.0


In [24]:
# Export the super_clean dataframe to a new CSV
filepath = Path('Data/Clean_Advertising.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
super_clean_df.to_csv(filepath, index=False)

PermissionError: [Errno 13] Permission denied: 'Data\\Clean_Advertising.csv'